In [1]:
!nvidia-smi


Mon Apr 17 22:55:02 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 531.41                 Driver Version: 531.41       CUDA Version: 12.1     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                      TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf            Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce GTX 1660 Ti    WDDM | 00000000:01:00.0 Off |                  N/A |
| N/A   43C    P8                1W /  N/A|      5MiB /  6144MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
import tensorflow as tf
print(tf.__version__)


2.11.0


In [4]:

from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob

In [5]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = './train_set/train/*'
valid_path = './train_set/val/*'

In [6]:
folders = glob('./train_set/train/*')
folders
len(folders)


2

In [7]:
# IMAGE_SIZE = [224, 224]
vgg16 = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


In [8]:
for layer in vgg16.layers:
    layer.trainable = False


In [9]:
x = Flatten()(vgg16.output)


In [10]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg16.input, outputs=prediction)

In [11]:
model.summary()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [12]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
# Use the Image Data Generator to import the images from the dataset
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [14]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory('./train_set/train/',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 3188 images belonging to 2 classes.


In [15]:
test_set = test_datagen.flow_from_directory('./train_set/test/',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 400 images belonging to 2 classes.


In [16]:
# fit the model
# Run the cell. It will take some time to execute
# r = model.fit_generator(
#   training_set,
#   validation_data=test_set,
#   epochs=5,
#   steps_per_epoch=len(training_set),
#   validation_steps=len(test_set)
# )

In [17]:
# fit the model
# Run the cell. It will take some time to execute
# r = model.fit(
#   training_set,
#   validation_data=test_set,
#   epochs=10,
#   steps_per_epoch=len(training_set),
#   validation_steps=len(test_set)
# )

In [18]:
# fit the model
# Run the cell. It will take some time to execute
# r = model.fit(
#   training_set,
#   validation_data=test_set,
#   epochs=15,
#   steps_per_epoch=len(training_set),
#   validation_steps=len(test_set)
# )

In [19]:
#fit the model
#Run the cell. It will take some time to execute
r = model.fit(
  training_set,
  validation_data=test_set,
  epochs=25,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

Epoch 1/25
100/100 [==============================] - 291s 3s/step - loss: 0.5426 - accuracy: 0.7491 - val_loss: 0.4115 - val_accuracy: 0.7825
Epoch 2/25
100/100 [==============================] - 291s 3s/step - loss: 0.4202 - accuracy: 0.8018 - val_loss: 0.3824 - val_accuracy: 0.8175
Epoch 3/25
100/100 [==============================] - 289s 3s/step - loss: 0.4577 - accuracy: 0.7895 - val_loss: 0.5142 - val_accuracy: 0.7475
Epoch 4/25
100/100 [==============================] - 291s 3s/step - loss: 0.3998 - accuracy: 0.8231 - val_loss: 0.4072 - val_accuracy: 0.8000
Epoch 5/25
100/100 [==============================] - 291s 3s/step - loss: 0.3920 - accuracy: 0.8143 - val_loss: 0.5179 - val_accuracy: 0.7625
Epoch 6/25
100/100 [==============================] - 289s 3s/step - loss: 0.4182 - accuracy: 0.8087 - val_loss: 0.4234 - val_accuracy: 0.7975
Epoch 7/25
100/100 [==============================] - 289s 3s/step - loss: 0.3810 - accuracy: 0.8228 - val_loss: 0.3595 - val_accuracy: 0.8375

In [ ]:
import matplotlib.pyplot as plt
# plot the loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')

In [ ]:
# save it as a h5 file


from tensorflow.keras.models import load_model

model.save('model_vgg16.h5')

In [ ]:
y_pred = model.predict(test_set)
y_pred


In [ ]:
import numpy as np
y_pred = np.argmax(y_pred, axis=1)

In [ ]:
y_pred


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [ ]:
model=load_model('model_resnet50.h5')


In [ ]:
img_data


In [ ]:
img=image.load_img('Datasets/Test/Coffee/download (2).jpg',target_size=(224,224))


In [ ]:
x=image.img_to_array(img)
x

In [ ]:
x.shape


In [ ]:
import numpy as np
x=np.expand_dims(x,axis=0)
img_data=preprocess_input(x)
img_data.shape

In [ ]:
model.predict(img_data)


In [ ]:
a=np.argmax(model.predict(img_data), axis=1)

In [ ]:
a==1